In [5]:
!nvidia-smi

Tue Mar 21 20:52:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661963 sha256=d422713ec672d215886272523322f41125ec5f9de54637fca15bf3480a7fdbca
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69866 sha256=8886f239d23c6b7f0a910d96be8d84b167fa2dccb61f76a67a02008078d088d4
  Stored in directory: /root/.cache/pip/wheels/76/ee/8

In [1]:
%%writefile test.py

import pycuda.autoinit
import pycuda.driver as drv
from pycuda import gpuarray
from pycuda.compiler import SourceModule

ker = SourceModule('''
__global__ void hello_world_ker()
{
	printf("Hello world from thread %d, in block %d!\\n", threadIdx.x, blockIdx.x);
	
	__syncthreads();
	
	if(threadIdx.x == 0 && blockIdx.x == 0)
	{
		printf("-------------------------------------\\n");
		printf("This kernel was launched over a grid consisting of %d blocks,\\n", gridDim.x);
		printf("where each block has %d threads.\\n", blockDim.x);
	}
}
''')

hello_ker = ker.get_function("hello_world_ker")
hello_ker(  block=(5,1,1), grid=(2,1,1) )

Overwriting test.py


In [2]:
!python test.py

Hello world from thread 0, in block 1!
Hello world from thread 1, in block 1!
Hello world from thread 2, in block 1!
Hello world from thread 3, in block 1!
Hello world from thread 4, in block 1!
Hello world from thread 0, in block 0!
Hello world from thread 1, in block 0!
Hello world from thread 2, in block 0!
Hello world from thread 3, in block 0!
Hello world from thread 4, in block 0!
-------------------------------------
This kernel was launched over a grid consisting of 2 blocks,
where each block has 5 threads.
